# 🧪 SchNet GNN: Molecular Property Prediction
This notebook demonstrates SchNet for predicting quantum energies using SMILES from PubChem/NIST.

In [ ]:
import torch
from torch_geometric.nn.models import SchNet
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_geometric.data import Data
import numpy as np
from ase.units import Hartree

# Example: H2O molecule
smiles = 'O'
energy_hartree = -76.0
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, AllChem.ETKDG())

# Build torch_geometric.Data (atom numbers, positions, target energy)
atom_types = []
positions = []
conf = mol.GetConformer()
for atom in mol.GetAtoms():
    atom_types.append(atom.GetAtomicNum())
    pos = conf.GetAtomPosition(atom.GetIdx())
    positions.append([pos.x, pos.y, pos.z])
x = torch.tensor(atom_types, dtype=torch.long).view(-1, 1)
pos = torch.tensor(positions, dtype=torch.float)
y = torch.tensor([energy_hartree * Hartree], dtype=torch.float)  # convert to eV

data = Data(z=x.view(-1), pos=pos, y=y)

# Build SchNet
model = SchNet(hidden_channels=64, num_filters=64, num_interactions=3, cutoff=10.0)
pred = model(data.z, data.pos, batch=None)
print('Predicted Energy (eV):', pred.item())